In [39]:
import requests
import os



# An api key is emailed to you when you sign up to a plan
# Get a free API key at https://api.the-odds-api.com/
API_KEY = os.getenv("API_KEY")

SPORT = 'soccer_epl' # use the sport_key from the /sports endpoint below, or use 'upcoming' to see the next 8 games across all sports
REGIONS = 'uk' # uk | us | eu | au. Multiple can be specified if comma delimited
MARKETS = 'h2h' # h2h | spreads | totals. Multiple can be specified if comma delimited
ODDS_FORMAT = 'decimal' # decimal | american
DATE_FORMAT = 'iso' # iso | unixzz


In [41]:
# # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # 
#
# First get a list of in-season sports
#   The sport 'key' from the response can be used to get odds in the next request
#
# # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # 

sports_response = requests.get(
    'https://api.the-odds-api.com/v4/sports', 
    params={
        'api_key': API_KEY
    }
)


if sports_response.status_code != 200:
    print(f'Failed to get sports: status_code {sports_response.status_code}, response body {sports_response.text}')

else:
    print('List of in season sports:', sports_response.json())



List of in season sports: [{'key': 'americanfootball_cfl', 'group': 'American Football', 'title': 'CFL', 'description': 'Canadian Football League', 'active': True, 'has_outrights': False}, {'key': 'americanfootball_ncaaf', 'group': 'American Football', 'title': 'NCAAF', 'description': 'US College Football', 'active': True, 'has_outrights': False}, {'key': 'americanfootball_ncaaf_championship_winner', 'group': 'American Football', 'title': 'NCAAF Championship Winner', 'description': 'US College Football Championship Winner', 'active': True, 'has_outrights': True}, {'key': 'americanfootball_nfl', 'group': 'American Football', 'title': 'NFL', 'description': 'US Football', 'active': True, 'has_outrights': False}, {'key': 'americanfootball_nfl_super_bowl_winner', 'group': 'American Football', 'title': 'NFL Super Bowl Winner', 'description': 'Super Bowl Winner 2025/2026', 'active': True, 'has_outrights': True}, {'key': 'baseball_kbo', 'group': 'Baseball', 'title': 'KBO', 'description': 'KBO 

In [42]:

# # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # 
#
# Now get a list of live & upcoming games for the sport you want, along with odds for different bookmakers
# This will deduct from the usage quota
# The usage quota cost = [number of markets specified] x [number of regions specified]
# For examples of usage quota costs, see https://the-odds-api.com/liveapi/guides/v4/#usage-quota-costs
#
# # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # 

odds_response = requests.get(
    f'https://api.the-odds-api.com/v4/sports/{SPORT}/odds',
    params={
        'api_key': API_KEY,
        'regions': REGIONS,
        'markets': MARKETS,
        'oddsFormat': ODDS_FORMAT,
        'dateFormat': DATE_FORMAT,
    }
)

if odds_response.status_code != 200:
    print(f'Failed to get odds: status_code {odds_response.status_code}, response body {odds_response.text}')

else:
    odds_json = odds_response.json()
    print('Number of events:', len(odds_json))
    print(odds_json)

    # Check the usage quota
    print('Remaining requests', odds_response.headers['x-requests-remaining'])
    print('Used requests', odds_response.headers['x-requests-used'])


Number of events: 17
[{'id': '354123128ddb07b54e61c8f0953029df', 'sport_key': 'soccer_epl', 'sport_title': 'EPL', 'commence_time': '2025-10-25T16:30:00Z', 'home_team': 'Manchester United', 'away_team': 'Brighton and Hove Albion', 'bookmakers': [{'key': 'paddypower', 'title': 'Paddy Power', 'last_update': '2025-10-25T16:44:53Z', 'markets': [{'key': 'h2h', 'last_update': '2025-10-25T16:44:53Z', 'outcomes': [{'name': 'Brighton and Hove Albion', 'price': 3.0}, {'name': 'Manchester United', 'price': 2.1}, {'name': 'Draw', 'price': 3.5}]}]}, {'key': 'smarkets', 'title': 'Smarkets', 'last_update': '2025-10-25T16:45:18Z', 'markets': [{'key': 'h2h', 'last_update': '2025-10-25T16:45:18Z', 'outcomes': [{'name': 'Brighton and Hove Albion', 'price': 3.3}, {'name': 'Manchester United', 'price': 2.3}, {'name': 'Draw', 'price': 3.5}]}, {'key': 'h2h_lay', 'last_update': '2025-10-25T16:45:18Z', 'outcomes': [{'name': 'Brighton and Hove Albion', 'price': 3.5}, {'name': 'Manchester United', 'price': 2.4}, 

In [ ]:
import pandas as pd

# Convert the odds data to a CSV format
# We'll flatten the nested structure to create a comprehensive CSV

rows = []

for event in odds_json:
    event_id = event['id']
    sport_key = event['sport_key']
    sport_title = event['sport_title']
    commence_time = event['commence_time']
    home_team = event['home_team']
    away_team = event['away_team']
    
    # Loop through each bookmaker
    for bookmaker in event['bookmakers']:
        bookmaker_key = bookmaker['key']
        bookmaker_title = bookmaker['title']
        bookmaker_last_update = bookmaker['last_update']
        
        # Loop through each market (h2h, spreads, etc.)
        for market in bookmaker['markets']:
            market_key = market['key']
            market_last_update = market['last_update']
            

            for outcome in market['outcomes']:
                row = {
                    'event_id': event_id,
                    'sport_key': sport_key,
                    'sport_title': sport_title,
                    'commence_time': commence_time,
                    'home_team': home_team,
                    'away_team': away_team,
                    'bookmaker_key': bookmaker_key,
                    'bookmaker_title': bookmaker_title,
                    'bookmaker_last_update': bookmaker_last_update,
                    'market_key': market_key,
                    'market_last_update': market_last_update,
                }
                rows.append(row)

# Create DataFrame
df = pd.DataFrame(rows)


In [45]:
print(df.head())

                           event_id   sport_key sport_title  \
0  354123128ddb07b54e61c8f0953029df  soccer_epl         EPL   
1  354123128ddb07b54e61c8f0953029df  soccer_epl         EPL   
2  354123128ddb07b54e61c8f0953029df  soccer_epl         EPL   
3  354123128ddb07b54e61c8f0953029df  soccer_epl         EPL   
4  354123128ddb07b54e61c8f0953029df  soccer_epl         EPL   

          commence_time          home_team                 away_team  \
0  2025-10-25T16:30:00Z  Manchester United  Brighton and Hove Albion   
1  2025-10-25T16:30:00Z  Manchester United  Brighton and Hove Albion   
2  2025-10-25T16:30:00Z  Manchester United  Brighton and Hove Albion   
3  2025-10-25T16:30:00Z  Manchester United  Brighton and Hove Albion   
4  2025-10-25T16:30:00Z  Manchester United  Brighton and Hove Albion   

  bookmaker_key bookmaker_title bookmaker_last_update market_key  \
0    paddypower     Paddy Power  2025-10-25T16:44:53Z        h2h   
1    paddypower     Paddy Power  2025-10-25T16:44:53